# MFA with mixed Data

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

In [2]:
# Chargement des données
import pandas as pd
Biometrie2 = pd.read_csv("./data/Biometrie2.csv",sep=";",header=0,index_col=0)
Biometrie2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, A to F
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Longueur3classes  6 non-null      object
 1   Poids2classes     6 non-null      object
 2   Largeur3classes   6 non-null      object
 3   Longueur          6 non-null      int64 
 4   Poids             6 non-null      int64 
 5   Largeur           6 non-null      int64 
 6   Long3cl           6 non-null      object
 7   Poids2cl          6 non-null      object
 8   Larg3cl           6 non-null      object
 9   Long              6 non-null      int64 
 10  Poid              6 non-null      int64 
 11  Larg              6 non-null      int64 
dtypes: int64(6), object(6)
memory usage: 624.0+ bytes


In [3]:
Biometrie2

,Longueur3classes,Poids2classes,Largeur3classes,Longueur,Poids,Largeur,Long3cl,Poids2cl,Larg3cl,Long,Poid,Larg
A,Long1,Poids2,Larg3,1,1,6,Long1,Poids2,Larg3,1,1,6
B,Long1,Poids1,Larg3,2,0,5,Long1,Poids1,Larg3,2,0,5
C,Long2,Poids2,Larg2,3,2,3,Long2,Poids2,Larg2,3,2,3
D,Long3,Poids1,Larg2,5,0,4,Long3,Poids1,Larg2,5,0,4
E,Long2,Poids2,Larg1,4,1,1,Long2,Poids2,Larg1,4,1,1
F,Long3,Poids1,Larg1,6,0,2,Long3,Poids1,Larg1,6,0,2


In [4]:
D = Biometrie2.iloc[:,:6]
group1 = [("quali",col) for col in D.columns.tolist()[:3]]
group2 = [("quanti",col) for col in D.columns.tolist()[3:]]
columns = [*group1,*group2]
D.columns = pd.MultiIndex.from_tuples(columns)

In [5]:
D.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, A to F
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   (quali, Longueur3classes)  6 non-null      object
 1   (quali, Poids2classes)     6 non-null      object
 2   (quali, Largeur3classes)   6 non-null      object
 3   (quanti, Longueur)         6 non-null      int64 
 4   (quanti, Poids)            6 non-null      int64 
 5   (quanti, Largeur)          6 non-null      int64 
dtypes: int64(3), object(3)
memory usage: 336.0+ bytes


In [6]:
from scientisttools.decomposition import MFA

res_mfa = MFA(n_components=None,group=["quali","quanti"],group_sup=None,row_labels=D.index,parallelize=True)
# Instanciation
res_mfa.fit(D)

MFA(group=['quali', 'quanti'], parallelize=True,
    row_labels=Index(['A', 'B', 'C', 'D', 'E', 'F'], dtype='object'))

## Eigenvalues

In [7]:
from scientisttools.extractfactor import get_eig
eig = get_eig(res_mfa)
eig.round(4)

,eigenvalue,difference,proportion,cumulative
Dim.1,1.9198,0.3905,45.7154,45.7154
Dim.2,1.5293,0.9221,36.4169,82.1323
Dim.3,0.6072,0.4894,14.4598,96.5921
Dim.4,0.1178,0.0926,2.8060,99.3980
Dim.5,0.0253,0.0253,0.6020,100.0000


## Separates analyses

In [8]:
res_mfa.separate_analyses_.keys()

dict_keys(['quali', 'quanti'])

## Individuals

In [9]:
from scientisttools.extractfactor import get_mfa_ind
ind = get_mfa_ind(res_mfa)
ind.keys()

dict_keys(['coord', 'contrib', 'cos2', 'coord_partiel', 'within_inertia', 'within_partial_inertia'])

### Coordonnées

In [10]:
ind["coord"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
A,-2.170061,0.040655,-0.209135,0.534032,-0.036846
B,-1.389024,-1.207001,-0.338205,-0.569141,0.051539
C,-0.064652,1.876511,0.916203,-0.122481,0.173123
D,0.898058,-1.106374,1.169873,0.035797,-0.195687
E,0.941581,1.400920,-0.985456,-0.131714,-0.195274
F,1.784097,-1.004711,-0.553280,0.253508,0.203145


### Cosinus carré

In [11]:
ind["cos2"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
A,0.934154,0.000328,0.008676,0.056573,0.000269
B,0.504125,0.380657,0.029887,0.084637,0.000694
C,0.000948,0.798502,0.190352,0.003402,0.006796
D,0.234535,0.355962,0.397994,0.000373,0.011136
E,0.228749,0.506373,0.250564,0.004476,0.009839
F,0.691341,0.219249,0.066488,0.013959,0.008963


### Contributions

In [12]:
ind["contrib"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
A,40.882362,0.018013,1.200462,40.337447,0.895049
B,16.749872,15.876970,3.139457,45.815779,1.751256
C,0.036287,38.375526,23.039720,2.121843,19.759957
D,7.001663,13.340013,37.563982,0.181248,25.246427
E,7.696759,21.388428,26.654359,2.453802,25.139985
F,27.633057,11.001051,8.402019,9.089880,27.207325


### Partial coordinates

In [13]:
ind["coord_partiel"]

quali                                            quanti            \
      Dim.1     Dim.2     Dim.3     Dim.4     Dim.5     Dim.1     Dim.2   
A -2.078405 -0.039441 -1.054187  0.493646 -0.196679 -2.261717  0.120751   
B -1.629355 -1.485879 -0.748551 -1.089564  1.359150 -1.148693 -0.928123   
C  0.220921  2.046547  1.507851 -0.654822 -1.662314 -0.350224  1.706474   
D  1.140130 -1.161634  2.378218  0.068163  0.479129  0.655987 -1.051114   
E  0.713750  1.924294 -1.476849  0.229796 -1.060365  1.169413  0.877546   
F  1.632959 -1.283887 -0.606481  0.952781  1.081078  1.935234 -0.725534   

                                 
      Dim.3     Dim.4     Dim.5  
A  0.635917  0.574417  0.122988  
B  0.072141 -0.048719 -1.256072  
C  0.324555  0.409860  2.008561  
D -0.038471  0.003432 -0.870504  
E -0.494063 -0.493224  0.669816  
F -0.500079 -0.445765 -0.674789

### Within Inertia

In [14]:
ind["within_inertia"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
A,2.988972,1.370763,24.314876,0.138620,0.360893
B,20.550490,16.617609,5.733308,23.019269,24.154642
C,29.015784,6.177679,11.918769,24.085708,47.590753
D,20.849154,0.652473,49.714942,0.089031,6.433015
E,18.468317,58.528202,8.221733,11.107624,10.572226
F,8.127283,16.653273,0.096372,41.559749,10.888472


### Whitin partial inertia

In [15]:
ind["within_partial_inertia"]

quali                                                 quanti  \
       Dim.1      Dim.2      Dim.3      Dim.4      Dim.5      Dim.1   
A   1.494486   0.685382  12.157438   0.069310   0.180446   1.494486   
B  10.275245   8.308805   2.866654  11.509634  12.077321  10.275245   
C  14.507892   3.088840   5.959385  12.042854  23.795377  14.507892   
D  10.424577   0.326236  24.857471   0.044515   3.216507  10.424577   
E   9.234159  29.264101   4.110866   5.553812   5.286113   9.234159   
F   4.063642   8.326637   0.048186  20.779874   5.444236   4.063642   

                                               
       Dim.2      Dim.3      Dim.4      Dim.5  
A   0.685382  12.157438   0.069310   0.180446  
B   8.308805   2.866654  11.509634  12.077321  
C   3.088840   5.959385  12.042854  23.795377  
D   0.326236  24.857471   0.044515   3.216507  
E  29.264101   4.110866   5.553812   5.286113  
F   8.326637   0.048186  20.779874   5.444236

## Quantitatives variables informations

In [16]:
from scientisttools.extractfactor import get_mfa_var
quanti_var = get_mfa_var(res_mfa,"quanti_var")
quanti_var.keys()

dict_keys(['coord', 'contrib', 'cos2', 'cor'])

### Coordinates

In [17]:
quanti_var["coord"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur,0.973325,-0.213091,0.056472,0.057280,0.027582
Poids,-0.219123,0.939266,0.183021,0.102501,0.160504
Largeur,-0.849243,-0.410004,0.299344,0.144856,-0.009662


### Contributions

In [18]:
quanti_var["contrib"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur,27.954367,1.681986,0.297512,1.577355,1.704785
Poids,1.416799,32.679188,3.124927,5.050956,57.728131
Largeur,21.281274,6.226869,8.359451,10.087676,0.209204


### Cos2

In [19]:
quanti_var["cos2"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur,0.947362,0.045408,0.003189,0.003281,0.000761
Poids,0.048015,0.882220,0.033497,0.010506,0.025761
Largeur,0.721213,0.168103,0.089607,0.020983,0.000093


## Qualitatives variables

In [20]:
quali_var = get_mfa_var(res_mfa,"quali_var")
quali_var.keys()

dict_keys(['coord', 'contrib', 'cos2', 'vtest', 'coord_partiel', 'within_inertia', 'within_partial_inertia'])

### Coordinates

In [21]:
quali_var["coord"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur3classes_Long1,-1.779542,-0.583173,-0.273670,-0.017555,0.007347
Longueur3classes_Long2,0.438465,1.638715,-0.034626,-0.127098,-0.011075
Longueur3classes_Long3,1.341078,-1.055542,0.308297,0.144653,0.003729
Poids2classes_Poids1,0.431044,-1.106029,0.092796,-0.093279,0.019666
Poids2classes_Poids2,-0.431044,1.106029,-0.092796,0.093279,-0.019666
Largeur3classes_Larg1,1.362839,0.198105,-0.769368,0.060897,0.003935
Largeur3classes_Larg2,0.416703,0.385068,1.043038,-0.043342,-0.011282
Largeur3classes_Larg3,-1.779542,-0.583173,-0.273670,-0.017555,0.007347


### Cos2

In [22]:
quali_var["cos2"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur3classes_Long1,0.884050,0.094941,0.020908,0.000086,0.000015
Longueur3classes_Long2,0.066405,0.927558,0.000414,0.005580,0.000042
Longueur3classes_Long3,0.593827,0.367877,0.031383,0.006909,0.000005
Poids2classes_Poids1,0.130221,0.857375,0.006035,0.006098,0.000271
Poids2classes_Poids2,0.130221,0.857375,0.006035,0.006098,0.000271
Largeur3classes_Larg1,0.745249,0.015747,0.237509,0.001488,0.000006
Largeur3classes_Larg2,0.122988,0.105023,0.770568,0.001331,0.000090
Largeur3classes_Larg3,0.884050,0.094941,0.020908,0.000086,0.000015


### Contributions

In [23]:
quali_var["contrib"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur3classes_Long1,14.320299,2.423542,3.385277,0.369914,1.407645
Longueur3classes_Long2,0.869371,19.136511,0.054195,19.389905,3.199092
Longueur3classes_Long3,8.132859,7.939754,4.296125,25.116160,0.362595
Poids2classes_Poids1,1.260287,13.076145,0.583834,15.665961,15.128778
Poids2classes_Poids2,1.260287,13.076145,0.583834,15.665961,15.128778
Largeur3classes_Larg1,8.398942,0.279670,26.755133,4.451348,0.403864
Largeur3classes_Larg2,0.785216,1.056650,49.174436,2.254851,3.319484
Largeur3classes_Larg3,14.320299,2.423542,3.385277,0.369914,1.407645


### V - test

In [24]:
quali_var["vtest"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur3classes_Long1,-2.030720,-0.745623,-0.555290,-0.080860,0.073060
Longueur3classes_Long2,0.500353,2.095199,-0.070259,-0.585424,-0.110140
Longueur3classes_Long3,1.530367,-1.349576,0.625549,0.666284,0.037080
Poids2classes_Poids1,0.695630,-1.999876,0.266279,-0.607618,0.276569
Poids2classes_Poids2,-0.695630,1.999876,-0.266279,0.607618,-0.276569
Largeur3classes_Larg1,1.555200,0.253289,-1.561085,0.280497,0.039134
Largeur3classes_Larg2,0.475520,0.492334,2.116376,-0.199637,-0.112194
Largeur3classes_Larg3,-2.030720,-0.745623,-0.555290,-0.080860,0.073060


### Coord partiel

In [25]:
quali_var["coord_partiel"]

quali                                          \
                           Dim.1     Dim.2     Dim.3     Dim.4     Dim.5   
Longueur3classes_Long1 -1.853880 -0.762660 -0.901369 -0.297959  0.581236   
Longueur3classes_Long2  0.467336  1.985421  0.015501 -0.212513 -1.361339   
Longueur3classes_Long3  1.386545 -1.222761  0.885868  0.510472  0.780104   
Poids2classes_Poids1    0.381245 -1.310467  0.341062 -0.022873  0.973119   
Poids2classes_Poids2   -0.381245  1.310467 -0.341062  0.022873 -0.973119   
Largeur3classes_Larg1   1.173355  0.320203 -1.041665  0.591288  0.010357   
Largeur3classes_Larg2   0.680525  0.442457  1.943034 -0.293330 -0.591593   
Largeur3classes_Larg3  -1.853880 -0.762660 -0.901369 -0.297959  0.581236   

                          quanti                                          
                           Dim.1     Dim.2     Dim.3     Dim.4     Dim.5  
Longueur3classes_Long1 -1.705205 -0.403686  0.354029  0.262849 -0.566542  
Longueur3classes_Long2  0.409594  1.292010 -0.084754 -0.041682  1.339188  
Longueur3classes_Long3  1.295611 -0.888324 -0.269275 -0.221166 -0.772646  
Poids2classes_Poids1    0.480843 -0.901590 -0.155470 -0.163684 -0.933788  
Poids2classes_Poids2   -0.480843  0.901590  0.155470  0.163684  0.933788  
Largeur3classes_Larg1   1.552323  0.076006 -0.497071 -0.469494 -0.002486  
Largeur3classes_Larg2   0.152881  0.327680  0.143042  0.206646  0.569029  
Largeur3classes_Larg3  -1.705205 -0.403686  0.354029  0.262849 -0.566542

### Whitin inertia

In [26]:
quali_var["within_inertia"]  # A vérifier

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Longueur3classes_Long1,4.441517,10.247029,18.484702,12.057491,6.285735
Longueur3classes_Long2,0.669928,38.234332,0.117885,1.118821,34.796676
Longueur3classes_Long3,1.661519,8.894075,15.650257,20.522106,11.503859
Poids2classes_Poids1,1.993223,13.294029,2.891636,0.760150,17.350007
Poids2classes_Poids2,1.993223,13.294029,2.891636,0.760150,17.350007
Largeur3classes_Larg1,28.857511,4.741917,3.478534,43.140231,0.000787
Largeur3classes_Larg2,55.941564,1.047560,38.000649,9.583560,6.427193
Largeur3classes_Larg3,4.441517,10.247029,18.484702,12.057491,6.285735


### Whitin partial inertia

In [27]:
quali_var["within_partial_inertia"] # A vérifier

quali                                              \
                            Dim.1      Dim.2      Dim.3      Dim.4      Dim.5   
Longueur3classes_Long1   2.220758   5.123514   9.242351   6.028746   3.142867   
Longueur3classes_Long2   0.334964  19.117166   0.058942   0.559410  17.398338   
Longueur3classes_Long3   0.830759   4.447037   7.825128  10.261053   5.751930   
Poids2classes_Poids1     0.996611   6.647015   1.445818   0.380075   8.675004   
Poids2classes_Poids2     0.996611   6.647015   1.445818   0.380075   8.675004   
Largeur3classes_Larg1   14.428755   2.370958   1.739267  21.570115   0.000394   
Largeur3classes_Larg2   27.970782   0.523780  19.000325   4.791780   3.213597   
Largeur3classes_Larg3    2.220758   5.123514   9.242351   6.028746   3.142867   

                           quanti                                              
                            Dim.1      Dim.2      Dim.3      Dim.4      Dim.5  
Longueur3classes_Long1   2.220758   5.123514   9.242351   6.028746   3.142867  
Longueur3classes_Long2   0.334964  19.117166   0.058942   0.559410  17.398338  
Longueur3classes_Long3   0.830759   4.447037   7.825128  10.261053   5.751930  
Poids2classes_Poids1     0.996611   6.647015   1.445818   0.380075   8.675004  
Poids2classes_Poids2     0.996611   6.647015   1.445818   0.380075   8.675004  
Largeur3classes_Larg1   14.428755   2.370958   1.739267  21.570115   0.000394  
Largeur3classes_Larg2   27.970782   0.523780  19.000325   4.791780   3.213597  
Largeur3classes_Larg3    2.220758   5.123514   9.242351   6.028746   3.142867

## Group

In [28]:
group = get_mfa_var(res_mfa,element="group")
group.keys()

dict_keys(['coord', 'correlation', 'contrib', 'cos2', 'dist', 'Lg', 'RV'])

### Coordinates

In [29]:
group["coord"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
quali,0.947376,0.908595,0.535689,0.098138,0.010202
quanti,0.972427,0.620718,0.071543,0.019697,0.015077


### Contribution

In [30]:
group["contrib"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
quali,49.34756,59.411958,88.21811,83.284013,40.357881
quanti,50.65244,40.588042,11.78189,16.715987,59.642119


### Correlation

In [31]:
group["correlation"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
quali,0.98779,0.994531,0.966165,0.833307,0.117382
quanti,0.98840,0.988391,0.447416,0.289588,0.172073


### Lg

In [32]:
group["Lg"]

,quali,quanti
quali,2.083333,1.462078
quanti,1.462078,1.400209


### RV

In [33]:
group["RV"]

,quali,quanti
quali,1.000000,0.856041
quanti,0.856041,1.000000


### Dist

In [34]:
group["dist"]

,dist
quali,2.083333
quanti,1.400209


### Cos2

In [35]:
group["cos2"]

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
quali,0.430810,0.396262,0.137742,0.004623,0.000050
quanti,0.675338,0.275167,0.003656,0.000277,0.000162


## Partial axes

In [36]:
from scientisttools.extractfactor import get_mfa_partial_axes
partial_axes = get_mfa_partial_axes(res_mfa)
partial_axes.keys()

dict_keys(['coord', 'cor', 'cos2', 'contrib', 'cor_between'])

### Coordinates

In [37]:
partial_axes["coord"]

quali                                            quanti            \
          Dim.1     Dim.2     Dim.3     Dim.4     Dim.5     Dim.1     Dim.2   
Dim.1 -0.908166 -0.302745  0.278772  0.074522 -0.017707  0.981189  0.113547   
Dim.2 -0.333453  0.935871 -0.061721 -0.008146  0.095283 -0.108399  0.983967   
Dim.3 -0.248333 -0.156734 -0.949516 -0.099987  0.046759 -0.148277  0.014467   
Dim.4 -0.036162 -0.026996  0.123970 -0.982030  0.134951 -0.058139  0.016093   
Dim.5  0.032673 -0.084834  0.039073  0.141419  0.984992 -0.012294  0.135825   

                 
          Dim.3  
Dim.1  0.149827  
Dim.2 -0.067310  
Dim.3  0.835411  
Dim.4  0.477605  
Dim.5  0.216802

### Correlation

In [38]:
partial_axes["cor"]

quali                                            quanti            \
          Dim.1     Dim.2     Dim.3     Dim.4     Dim.5     Dim.1     Dim.2   
Dim.1 -0.908166 -0.302745  0.278772  0.074522 -0.017707  0.981189  0.113547   
Dim.2 -0.333453  0.935871 -0.061721 -0.008146  0.095283 -0.108399  0.983967   
Dim.3 -0.248333 -0.156734 -0.949516 -0.099987  0.046759 -0.148277  0.014467   
Dim.4 -0.036162 -0.026996  0.123970 -0.982030  0.134951 -0.058139  0.016093   
Dim.5  0.032673 -0.084834  0.039073  0.141419  0.984992 -0.012294  0.135825   

                 
          Dim.3  
Dim.1  0.149827  
Dim.2 -0.067310  
Dim.3  0.835411  
Dim.4  0.477605  
Dim.5  0.216802

### Cor between

In [39]:
partial_axes["cor_between"]

,Dim.1_quali,Dim.2_quali,Dim.3_quali,Dim.4_quali,Dim.5_quali,Dim.1_quanti,Dim.2_quanti,Dim.3_quanti
Dim.1_quali,1.000000e+00,0.000000e+00,-3.925231e-17,-3.436168e-17,-9.125092e-17,-8.164134e-01,-4.309624e-01,-3.312710e-01
Dim.2_quali,0.000000e+00,1.000000e+00,0.000000e+00,4.807407e-17,0.000000e+00,-3.726444e-01,8.722657e-01,-2.705756e-01
Dim.3_quali,-3.925231e-17,0.000000e+00,1.000000e+00,-3.239651e-17,-6.452414e-17,4.133229e-01,-3.551181e-02,-6.796345e-01
Dim.4_quali,-3.436168e-17,4.807407e-17,-3.239651e-17,1.000000e+00,5.648478e-17,1.441848e-01,2.405005e-03,-5.101789e-01
Dim.5_quali,-9.125092e-17,0.000000e+00,-6.452414e-17,5.648478e-17,1.000000e+00,-5.459134e-02,2.283793e-01,3.079979e-01
Dim.1_quanti,-8.164134e-01,-3.726444e-01,4.133229e-01,1.441848e-01,-5.459134e-02,1.000000e+00,-5.288191e-17,-6.893053e-17
Dim.2_quanti,-4.309624e-01,8.722657e-01,-3.551181e-02,2.405005e-03,2.283793e-01,-5.288191e-17,1.000000e+00,1.894004e-16
Dim.3_quanti,-3.312710e-01,-2.705756e-01,-6.796345e-01,-5.101789e-01,3.079979e-01,-6.893053e-17,1.894004e-16,1.000000e+00


## Inertia ratio

In [40]:
res_mfa.inertia_ratio_

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
Inertia ratio,0.976181,0.95147,0.553679,0.375353,0.020978


In [41]:
X = res_mfa.active_data_
res_mfa.transform(D)

,Dim.1,Dim.2,Dim.3,Dim.4,Dim.5
A,-2.170061,0.040655,-0.209135,0.534032,-0.036846
B,-1.389024,-1.207001,-0.338205,-0.569141,0.051539
C,-0.064652,1.876511,0.916203,-0.122481,0.173123
D,0.898058,-1.106374,1.169873,0.035797,-0.195687
E,0.941581,1.400920,-0.985456,-0.131714,-0.195274
F,1.784097,-1.004711,-0.553280,0.253508,0.203145
